# OpenVINO

[OpenVINO™](https://github.com/openvinotoolkit/openvino) 是一个用于优化和部署 AI 推理的开源工具包。OpenVINO™ Runtime 能够支持在各种硬件 [设备](https://github.com/openvinotoolkit/openvino?tab=readme-ov-file#supported-hardware-matrix) 上运行经过优化的相同模型。在语言 + LLM、计算机视觉、自动语音识别等用例中加速您的深度学习性能。

可以通过 `HuggingFacePipeline` [类](https://python.langchain.com/docs/integrations/llms/huggingface_pipeline) 在本地运行 OpenVINO 模型。要使用 OpenVINO 部署模型，您可以指定 `backend="openvino"` 参数来将 OpenVINO 触发为后端推理框架。

要使用，您应该安装 ``optimum-intel`` 的 OpenVINO 加速器 Python [程序包](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#installation)。

In [ ]:
%pip install --upgrade-strategy eager "optimum[openvino,nncf]" langchain-huggingface --quiet

### 模型加载

模型可以通过指定 `from_model_id` 方法的模型参数来加载。

如果您的设备是 Intel GPU，您可以指定 `model_kwargs={"device": "GPU"}` 以在 GPU 上运行推理。

In [ ]:
from langchain_huggingface import HuggingFacePipeline

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

ov_llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    backend="openvino",
    model_kwargs={"device": "CPU", "ov_config": ov_config},
    pipeline_kwargs={"max_new_tokens": 10},
)

它们也可以通过传入现有的 [`optimum-intel`](https://huggingface.co/docs/optimum/main/en/intel/inference) pipeline 来加载

In [ ]:
from optimum.intel.openvino import OVModelForCausalLM
from transformers import AutoTokenizer, pipeline

model_id = "gpt2"
device = "CPU"
tokenizer = AutoTokenizer.from_pretrained(model_id)
ov_model = OVModelForCausalLM.from_pretrained(
    model_id, export=True, device=device, ov_config=ov_config
)
ov_pipe = pipeline(
    "text-generation", model=ov_model, tokenizer=tokenizer, max_new_tokens=10
)
ov_llm = HuggingFacePipeline(pipeline=ov_pipe)

### 创建链

将模型加载到内存后，您可以将其与提示组合以形成链。

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate.from_template(template)

chain = prompt | ov_llm

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

要获取无提示的响应，您可以将 `skip_prompt=True` 绑定到 LLM。

In [ ]:
chain = prompt | ov_llm.bind(skip_prompt=True)

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

### 使用本地 OpenVINO 模型进行推理

您可以使用 CLI 将模型[导出为 OpenVINO IR 格式](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#export)，并从本地文件夹加载模型。

In [ ]:
!optimum-cli export openvino --model gpt2 ov_model_dir

建议应用 8 位或 4 位权重量化，使用 `--weight-format` 来降低推理延迟和模型占用空间：

In [ ]:
!optimum-cli export openvino --model gpt2  --weight-format int8 ov_model_dir # for 8-bit quantization

!optimum-cli export openvino --model gpt2  --weight-format int4 ov_model_dir # for 4-bit quantization

In [ ]:
ov_llm = HuggingFacePipeline.from_model_id(
    model_id="ov_model_dir",
    task="text-generation",
    backend="openvino",
    model_kwargs={"device": "CPU", "ov_config": ov_config},
    pipeline_kwargs={"max_new_tokens": 10},
)

chain = prompt | ov_llm

question = "What is electroencephalography?"

print(chain.invoke({"question": question}))

您可以通过激活和 KV 缓存量化实现额外的推理速度提升。这些选项可以通过 `ov_config` 启用，如下所示：

In [ ]:
ov_config = {
    "KV_CACHE_PRECISION": "u8",
    "DYNAMIC_QUANTIZATION_GROUP_SIZE": "32",
    "PERFORMANCE_HINT": "LATENCY",
    "NUM_STREAMS": "1",
    "CACHE_DIR": "",
}

### 流式输出

你可以使用 `stream` 方法来获得 LLM 输出的流式传输。

In [ ]:
generation_config = {"skip_prompt": True, "pipeline_kwargs": {"max_new_tokens": 100}}
chain = prompt | ov_llm.bind(**generation_config)

for chunk in chain.stream(question):
    print(chunk, end="", flush=True)

更多信息请参阅：

* [OpenVINO LLM 指南](https://docs.openvino.ai/2024/learn-openvino/llm_inference_guide.html)。

* [OpenVINO 文档](https://docs.openvino.ai/2024/home.html)。

* [OpenVINO 入门指南](https://www.intel.com/content/www/us/en/content-details/819067/openvino-get-started-guide.html)。
  
* [使用 LangChain 的 RAG Notebook](https://github.com/openvinotoolkit/openvino_notebooks/tree/latest/notebooks/llm-rag-langchain)。